# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 4

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,22,1997,3,2,192,1,0,0,0,0,0,0.47,0.00,0.47,0.47,0.47,1.7,1.7,0.0,0.78,0.01,0.79,0.78,0.79,Matches
2,Che Adams\Che-Adams,eng ENG,FW,Southampton,24,1996,4,4,322,0,1,0,0,1,0,0.00,0.28,0.28,0.00,0.28,1.1,1.1,0.6,0.31,0.17,0.48,0.31,0.48,Matches
3,Adrián\Adrian,es ESP,GK,Liverpool,33,1987,1,1,90,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches
4,Ola Aina\Ola-Aina,ng NGA,DF,Fulham,23,1996,1,1,90,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.03,0.03,0.00,0.03,Matches
5,Semi Ajayi\Semi-Ajayi,ng NGA,DF,West Brom,26,1993,4,4,360,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col='Squad')
teamStats.head()

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Squad,,,,,,,,,,,,,,,,,,
Everton,1,4,4,0,0,12,5,7,12,8.6,2.9,5.7,1.41,W W W W,NaN,Dominic Calvert-Lewin - 6,Jordan Pickford,NaN
Aston Villa,2,3,3,0,0,11,2,9,9,6.6,3.3,3.2,1.07,W W W,NaN,Ollie Watkins Jack Grealish - 3,Emiliano Martínez,NaN
Leicester City,3,4,3,0,1,12,7,5,9,6.6,4.9,1.7,0.41,W W W L,NaN,Jamie Vardy - 5,Kasper Schmeichel,NaN
Arsenal,4,4,3,0,1,8,5,3,9,4.7,5.3,-0.6,-0.14,W W L W,NaN,Alexandre Lacazette - 3,Bernd Leno,NaN
Liverpool,5,4,3,0,1,11,11,0,9,10.6,6.7,3.9,0.98,W W W L,NaN,Mohamed Salah - 5,Alisson,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)
# this is used later
df_online=1

# fetch data locally
#df = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek) + '.csv', index_col=0)
#df_online=0

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,NaN,NaN,0,37605,NaN,,0,0,-2,2,0.0,568,232,NaN,,0,3,-0.5,0.5,0,Mesut,0.0,0,0,0.0,568,232,False,0.0,567,232,0,,None,68,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.8,False,None,a,1,3,0.0,567,231,0,2760,443,39534,3614,0.0,0.0,Özil,0
2,0,0,0,100.0,75.0,0,39476,NaN,,0,0,-1,1,0.0,476,188,NaN,,0,2,-0.5,0.4,0,Sokratis,0.0,0,0,0.0,488,188,False,0.0,484,188,0,,2020-09-11T11:00:08.600094Z,49,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,None,a,1,3,0.0,449,177,0,9919,157,16445,288,0.0,0.0,Sokratis,0
3,0,0,27,100.0,100.0,0,41270,NaN,,0,0,0,0,4.0,255,81,4.0,,0,2,0.5,1.5,2,David,1.0,4,0,4.7,257,86,False,43.2,164,64,181,,2020-09-10T14:30:11.632386Z,55,0,0,NaN,0,,41270.jpg,1.3,0,0,Luiz Moreira Marinho,1.0,False,None,a,1,3,0.0,537,207,4,18215,2890,40948,4143,0.2,0.7,David Luiz,0
4,1,0,54,75.0,NaN,1,54694,NaN,,0,0,-1,1,57.2,60,44,6.0,,0,3,2.6,4.5,2,Pierre-Emerick,4.0,5,1,23.8,41,22,False,78.2,69,26,360,Ankle injury - 75% chance of playing,2020-10-08T09:00:12.305506Z,119,0,0,1.0,0,,54694.jpg,4.0,0,0,Aubameyang,24.3,False,None,d,1,3,103.0,30,13,16,324647,20449,1753242,284752,0.3,1.3,Aubameyang,1
5,0,0,0,100.0,100.0,0,58822,NaN,,0,0,-2,2,0.0,468,186,NaN,,0,2,-0.5,0.5,0,Cédric,0.0,0,0,0.0,482,186,False,0.0,478,186,0,,2020-09-23T09:00:14.881983Z,48,0,0,NaN,0,,58822.jpg,0.0,0,0,Soares,0.2,False,None,a,1,3,0.0,437,174,0,1951,383,26584,1095,0.0,0.0,Cédric,0


In [6]:
# assign proper team names for each player
team_names = np.sort(teamStats.index)
# for some reason the fpl team numbers are not alphabetical with leeds and leicester...
if (team_names[8]=='Leeds United') & (team_names[9]=='Leicester City'):
    team_names[8] = 'Leicester City'
    team_names[9] = 'Leeds United'
df['team_name'] = team_names[df['team']-1]

## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [7]:
teamStats['xG per game'] = teamStats['xG'] / teamStats['MP']
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Everton,1,4,4,0,0,12,5,7,12,8.6,2.9,5.7,1.41,W W W W,NaN,Dominic Calvert-Lewin - 6,Jordan Pickford,NaN,2.150000,0.725000,0.484325
Aston Villa,2,3,3,0,0,11,2,9,9,6.6,3.3,3.2,1.07,W W W,NaN,Ollie Watkins Jack Grealish - 3,Emiliano Martínez,NaN,2.200000,1.100000,0.332871
Leicester City,3,4,3,0,1,12,7,5,9,6.6,4.9,1.7,0.41,W W W L,NaN,Jamie Vardy - 5,Kasper Schmeichel,NaN,1.650000,1.225000,0.293758
Arsenal,4,4,3,0,1,8,5,3,9,4.7,5.3,-0.6,-0.14,W W L W,NaN,Alexandre Lacazette - 3,Bernd Leno,NaN,1.175000,1.325000,0.265803
Liverpool,5,4,3,0,1,11,11,0,9,10.6,6.7,3.9,0.98,W W W L,NaN,Mohamed Salah - 5,Alisson,NaN,2.650000,1.675000,0.187308
Tottenham,6,4,2,1,1,12,5,7,7,9.5,4.9,4.7,1.17,L W D W,NaN,Son Heung-min - 6,Hugo Lloris,NaN,2.375000,1.225000,0.293758
Chelsea,7,4,2,1,1,10,6,4,7,6.9,5.1,1.9,0.47,W L D W,NaN,Jorginho - 3,Kepa Arrizabalaga,NaN,1.725000,1.275000,0.279431
Leeds United,8,4,2,1,1,9,8,1,7,5.5,8.2,-2.7,-0.67,L W W D,NaN,Patrick Bamford - 3,Illan Meslier,NaN,1.375000,2.050000,0.128735
Newcastle Utd,9,4,2,1,1,6,5,1,7,5.0,5.6,-0.6,-0.15,W L D W,NaN,Callum Wilson - 4,Karl Darlow,NaN,1.250000,1.400000,0.246597


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players' assists based on their xA.

In [8]:
# auxiliary information and variables
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
xA_proportion

0.5330578512396694

In [9]:
df_previous_week = pd.read_csv('../data/fpl/data_week' + str(latest_gameweek-1) + '.csv', index_col=0)
if latest_gameweek == 1:
    df_previous_week['minutes'] = 0
    df_previous_week['xG'] = np.nan
    df_previous_week['xA'] = np.nan
df_previous_week.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,team_name,games played,adjusted points,xG,xG_points,xG_week3,goals_week3,xPoints week 3,clean_sheet_points,xGA_week3,cleansheet_week3,xA,xA_points,xA_week3,assists_week3,adjusted points per game,xPoints week 1,xPoints week 2,next1_xP,next5_xP,next10_xP,value,valuePoints metric,xG_week1,goals_week1,xGA_week1,cleansheet_week1,xA_week1,assists_week1,xG_week2,goals_week2,xGA_week2,cleansheet_week2,xA_week2,assists_week2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,NaN,NaN,0,37605,4.0,NaN,-1,1,-2,2,0.0,549,225,NaN,NaN,0,3,0.5,-0.5,0,Mesut,0.0,0,0,0.0,549,225,False,0.0,548,225,0,NaN,NaN,68,0,0,NaN,0,NaN,37605.jpg,0.0,0,0,Özil,0.8,False,...,a,1,3,0.0,548,224,0,2086,416,33139,3190,0.0,0.0,Özil,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,75.0,75.0,0,39476,NaN,NaN,0,0,-1,1,0.0,444,174,NaN,NaN,0,2,0.4,-0.4,0,Sokratis,0.0,0,0,0.0,458,175,False,0.0,453,175,0,Calf injury - 75% chance of playing,2020-09-11T11:00:08.600094Z,49,0,0,NaN,0,NaN,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,d,1,3,0.0,415,161,0,9750,26,15822,442,0.0,0.0,Sokratis,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,14,100.0,100.0,0,41270,NaN,NaN,0,0,0,0,3.2,233,67,4.0,NaN,0,2,1.2,0.2,1,David,0.7,3,0,2.3,270,93,False,20.2,209,84,91,NaN,2020-09-10T14:30:11.632386Z,55,0,0,NaN,0,NaN,41270.jpg,1.0,0,0,Luiz Moreira Marinho,1.0,False,...,a,1,3,0.0,509,192,2,12317,3172,34453,2564,0.1,0.4,David Luiz,0,Arsenal,2.000000,3.461468,0.0,0.0,0.0,0.0,1.199148,1.461468,3.0,0.0,0.0,0.0,0.0,0.0,1.730734,NaN,1.00000,2.167292,1.699840,1.741253,0.314679,0.737987,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.9,0.0,0.0,0.0
4,1,0,44,NaN,NaN,1,54694,NaN,NaN,0,0,-1,1,52.6,39,27,6.0,NaN,0,3,5.2,4.2,2,Pierre-Emerick,4.7,4,1,19.4,35,19,False,68.8,59,25,270,NaN,NaN,119,0,0,1.0,0,NaN,54694.jpg,4.7,0,0,Aubameyang,29.2,False,...,a,1,3,73.0,29,14,14,230759,33112,1353837,112065,0.4,1.2,Aubameyang,1,Arsenal,2.978723,10.940715,0.5,2.5,0.0,0.0,2.049787,0.544164,3.0,0.0,0.5,1.5,0.0,0.0,3.672954,3.818731,5.61799,3.780792,3.710141,3.761735,0.308652,1.064736,0.4,1.0,0.2,1.0,0.0,0.0,0.1,0.0,1.9,0.0,0.5,1.0
5,0,0,0,75.0,75.0,0,58822,NaN,NaN,-1,1,-2,2,0.0,438,171,NaN,NaN,0,2,0.4,-0.4,0,Cédric,0.0,0,0,0.0,453,172,False,0.0,448,172,0,Calf injury - 75% chance of playing,2020-09-23T09:00:14.881983Z,48,0,0,NaN,0,NaN,58822.jpg,0.0,0,0,Soares,0.3,False,...,d,1,3,0.0,408,158,0,1383,33,23726,2304,0.0,0.0,Cédric,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
if latest_gameweek > 1:
    filepath = '../data/fbref/team_stats_week' + str(latest_gameweek-1) + '.csv'
    teamStats_previous_week = pd.read_csv(filepath, index_col='Squad')
    display(teamStats_previous_week.head())
elif latest_gameweek == 1:
    teamStats_previous_week = pd.DataFrame(np.nan, index=team_names, columns=['xGA'])
    display(teamStats_previous_week)

,Rk,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xG per game,xGA per game,probability no goals allowed
Squad,,,,,,,,,,,,,,,,,,,,,
Leicester City,1,3,3,0,0,12,4,8,9,6.1,3.3,2.8,0.93,W W W,NaN,Jamie Vardy - 5,Kasper Schmeichel,NaN,2.033333,1.100000,0.332871
Liverpool,2,3,3,0,0,9,4,5,9,8.9,2.8,6.1,2.03,W W W,NaN,Sadio Mané Mohamed Salah - 3,Alisson,NaN,2.966667,0.933333,0.393241
Everton,3,3,3,0,0,8,3,5,9,6.7,1.7,5.0,1.68,W W W,NaN,Dominic Calvert-Lewin - 5,Jordan Pickford,NaN,2.233333,0.566667,0.567414
Aston Villa,4,2,2,0,0,4,0,4,6,2.6,1.6,1.1,0.54,W W,NaN,Ezri Konsa Jack Grealish... - 1,Emiliano Martínez,NaN,1.300000,0.800000,0.449329
Arsenal,5,3,2,0,1,6,4,2,6,4.4,5.1,-0.7,-0.25,W W L,NaN,Alexandre Lacazette - 3,Bernd Leno,NaN,1.466667,1.700000,0.182684


In [11]:
def points_for_goal(position):
    if position < 3:
        return 6
    elif position==3:
        return 5
    elif position==4:
        return 4

def points_for_clean_sheet(position):
    if position < 3:
        return 4
    elif position==3:
        return 1
    elif position==4:
        return 0

In [12]:
# column names to be used below
xPoints_column = 'xPoints week ' + str(latest_gameweek)
xG_column = 'xG_week' + str(latest_gameweek) 
goals_column = 'goals_week' + str(latest_gameweek) 
xGA_column = 'xGA_week' + str(latest_gameweek) 
cleansheet_column = 'cleansheet_week' + str(latest_gameweek) 
xA_column = 'xA_week' + str(latest_gameweek) 
assists_column = 'assists_week' + str(latest_gameweek) 

In [13]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    position = df.loc[ix, 'element_type']
    
    # 0.24 is the probability to miss a penalty, which incurs -2 points
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] \
                                        - points_for_goal(position) * (df.loc[ix, 'goals_scored'] - xG) \
                                        - 2 * 0.24 * penalty_attempts    
    df.loc[ix, 'xG'] = xG
    df.loc[ix, 'xG_points'] = points_for_goal(position)*xG - 2 * 0.24 * penalty_attempts
    
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xG']):
                df.loc[ix, xG_column] = df.loc[ix, 'xG'] - df_previous_week.loc[ix, 'xG']
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored'] - df_previous_week.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
            else:
                df.loc[ix, xG_column] = xG
                df.loc[ix, goals_column] = df.loc[ix, 'goals_scored']
                df.loc[ix, xPoints_column] = df.loc[ix, 'event_points'] - points_for_goal(position) * \
                                                        (df.loc[ix, goals_column] - df.loc[ix, xG_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xG)')

In [14]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[team, 'probability no goals allowed']
    position = df.loc[ix, 'element_type']
    
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] - points_for_clean_sheet(position) * \
                                ( df.loc[ix, 'clean_sheets'] - probability_cleanSheet*df.loc[ix, 'games played'] )
    df.loc[ix, 'clean_sheet_points'] = points_for_clean_sheet(position) * \
                                       df.loc[ix, 'games played'] * probability_cleanSheet
    try:
        if ~np.isnan(teamStats_previous_week.loc[team, 'xGA']):            
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA'] - teamStats_previous_week.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets'] - df_previous_week.loc[ix, 'clean_sheets']
        else:
            df.loc[ix, xGA_column] = teamStats.loc[team, 'xGA']
            df.loc[ix, cleansheet_column] = df.loc[ix, 'clean_sheets']
        
        if df.loc[ix,'minutes'] - df_previous_week.loc[ix,'minutes'] >= 60:
            probability_cleansheet_thisweek = poisson.pmf(0,df.loc[ix, xGA_column])
            df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - points_for_clean_sheet(position) * \
                                            (df.loc[ix, cleansheet_column] - probability_cleansheet_thisweek)
        
    except KeyError:  
        print(str(ix) + ' is a new index. (xGA)')

In [15]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)
    df.loc[ix, 'xA'] = xA
    df.loc[ix, 'xA_points'] = 3*xA
    try:
        if df.loc[ix,'minutes'] > df_previous_week.loc[ix,'minutes']:
            if ~np.isnan(df_previous_week.loc[ix, 'xA']):
                df.loc[ix, xA_column] = df.loc[ix, 'xA'] - df_previous_week.loc[ix, 'xA']
                df.loc[ix, assists_column] = df.loc[ix, 'assists'] - df_previous_week.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                    3 * (xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
            else:
                df.loc[ix, xA_column] = xA
                df.loc[ix, assists_column] = df.loc[ix, 'assists']
                df.loc[ix, xPoints_column] = df.loc[ix, xPoints_column] - \
                                            3*(xA_proportion*df.loc[ix, assists_column] - df.loc[ix, xA_column])
    except KeyError:  
        print(str(ix) + ' is a new index. (xA)')

In [16]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [17]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','elliott','mattylongstaff']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'mattylongstaff':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.contains('matthew longstaff'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

557 is a new index. (xG)
557 is a new index. (xGA)
557 is a new index. (xA)
562 is a new index. (xG)
562 is a new index. (xGA)
562 is a new index. (xA)
556 is a new index. (xG)
556 is a new index. (xGA)
556 is a new index. (xA)
555 is a new index. (xG)
555 is a new index. (xGA)
555 is a new index. (xA)
558 is a new index. (xG)
558 is a new index. (xGA)
558 is a new index. (xA)


In [18]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

if (latest_gameweek > 1) & (df_online==1):
    column_list = []
    for i in range(1,latest_gameweek):
        column_list.append('xPoints week ' + str(i))
        column_list.append('xG_week' + str(i))
        column_list.append('goals_week' + str(i))
        column_list.append('xGA_week' + str(i))
        column_list.append('cleansheet_week' + str(i))
        column_list.append('xA_week' + str(i))
        column_list.append('assists_week' + str(i))
    df = df.join( df_previous_week[column_list])

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,direct_freekicks_order,direct_freekicks_text,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_order,penalties_saved,penalties_text,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,...,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards,team_name,games played,adjusted points,xG,xG_points,xG_week4,goals_week4,xPoints week 4,clean_sheet_points,xGA_week4,cleansheet_week4,xA,xA_points,xA_week4,assists_week4,adjusted points per game,xPoints week 1,xG_week1,goals_week1,xGA_week1,cleansheet_week1,xA_week1,assists_week1,xPoints week 2,xG_week2,goals_week2,xGA_week2,cleansheet_week2,xA_week2,assists_week2,xPoints week 3,xG_week3,goals_week3,xGA_week3,cleansheet_week3,xA_week3,assists_week3
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,NaN,NaN,0,37605,NaN,,0,0,-2,2,0.0,568,232,NaN,,0,3,-0.5,0.5,0,Mesut,0.0,0,0,0.0,568,232,False,0.0,567,232,0,,None,68,0,0,NaN,0,,37605.jpg,0.0,0,0,Özil,0.8,False,...,3,0.0,567,231,0,2760,443,39534,3614,0.0,0.0,Özil,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,0,100.0,75.0,0,39476,NaN,,0,0,-1,1,0.0,476,188,NaN,,0,2,-0.5,0.4,0,Sokratis,0.0,0,0,0.0,488,188,False,0.0,484,188,0,,2020-09-11T11:00:08.600094Z,49,0,0,NaN,0,,39476.jpg,0.0,0,0,Papastathopoulos,0.1,False,...,3,0.0,449,177,0,9919,157,16445,288,0.0,0.0,Sokratis,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,27,100.0,100.0,0,41270,NaN,,0,0,0,0,4.0,255,81,4.0,,0,2,0.5,1.5,2,David,1.0,4,0,4.7,257,86,False,43.2,164,64,181,,2020-09-10T14:30:11.632386Z,55,0,0,NaN,0,,41270.jpg,1.3,0,0,Luiz Moreira Marinho,1.0,False,...,3,0.0,537,207,4,18215,2890,40948,4143,0.2,0.7,David Luiz,0,Arsenal,3.076923,7.271421,0.0,0.0,0.0,0.0,5.274923,3.271421,0.2,0.0,0.0,0.0,0.0,0.0,2.363212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,0.0,0.0,1.9,0.0,0.0,0.0,1.199148,0.0,0.0,3.0,0.0,0.0,0.0
4,1,0,54,75.0,NaN,1,54694,NaN,,0,0,-1,1,57.2,60,44,6.0,,0,3,2.6,4.5,2,Pierre-Emerick,4.0,5,1,23.8,41,22,False,78.2,69,26,360,Ankle injury - 75% chance of playing,2020-10-08T09:00:12.305506Z,119,0,0,1.0,0,,54694.jpg,4.0,0,0,Aubameyang,24.3,False,...,3,103.0,30,13,16,324647,20449,1753242,284752,0.3,1.3,Aubameyang,1,Arsenal,4.000000,13.464038,0.5,2.5,0.0,0.0,2.818731,1.063212,0.2,0.0,0.5,1.5,0.0,0.0,3.366010,3.818731,0.4,1.0,0.2,1.0,0.0,0.0,5.61799,0.1,0.0,1.9,0.0,0.5,1.0,2.049787,0.0,0.0,3.0,0.0,0.0,0.0
5,0,0,0,100.0,100.0,0,58822,NaN,,0,0,-2,2,0.0,468,186,NaN,,0,2,-0.5,0.5,0,Cédric,0.0,0,0,0.0,482,186,False,0.0,478,186,0,,2020-09-23T09:00:14.881983Z,48,0,0,NaN,0,,58822.jpg,0.0,0,0,Soares,0.2,False,...,3,0.0,437,174,0,1951,383,26584,1095,0.0,0.0,Cédric,0,Arsenal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
388,Kane,4.000000,44,11.0,43.824959,10.956240
389,Aurier,1.000000,12,12.0,9.175857,9.175857
232,Chilwell,1.000000,18,18.0,9.018550,9.018550
251,Mané,3.012048,25,8.3,26.064181,8.653308
37,Grealish,2.991453,35,11.7,25.398248,8.490271
254,Salah,4.019608,41,10.2,33.193732,8.257953
105,Jorginho,3.000000,27,9.0,24.719119,8.239706
506,Wilson,4.000000,34,8.5,30.440826,7.610207
508,Rodríguez,3.977273,35,8.8,29.427944,7.399026


In [20]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [21]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total xG: ' + str(total_xG))
print('Total goals: ' + str(total_goals))
print('Total xG / total goals: ' + str(total_xG/total_goals))

Total xG: 107.7
Total goals: 141
Total xG / total goals: 0.7638297872340426
